In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Mount Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
ranking = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/event_user_pages.csv')

In [ ]:
ranking.head()

,Unnamed: 0,Unnamed: 0.1,id_x,distinct_id,current_url,name,initial_referrerTrue,referrer,user_agent,performed_at,properties,inserted_at_x,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,id_y,url,published_at,title,dom_count,image_count,link_count,scripts_count,inserted_at_y,updated_at
0,0,0,28,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,https://byo.com/recipe/old-fiddlehead-barleywine/,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00 AM,"{""title"": ""Old Fiddlehead Maple Barleywine"", ""...",2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,36ccd600-ff57-4f1e-98ef-5b9804b7f331,https://byo.com/recipe/old-fiddlehead-barleywine/,NaN,Old Fiddlehead Maple Barleywine - Brew Your Own,521.0,9.0,83.0,50.0,9/26/20 19:10,9/26/20 19:10
1,1,1,37,adcc583b-c354-4e5d-ba77-21aa57362a57,https://byo.com/landing-page/digital-membershi...,One Page Checkout,https://www.google.com/,https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,"{""coupon"": """", ""product_id"": ""69709"", ""product...",2019/10/24 6:17:57 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,81aad24f-1bdb-4c96-8552-a1f49f1a4af6,https://byo.com/landing-page/digital-membershi...,NaN,Digital Membership + 14 days Free Trial - Brew...,540.0,16.0,26.0,61.0,12/10/20 18:37,12/10/20 18:37
2,2,2,49,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,https://byo.com/article/maple-beer/,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01 AM,"{""title"": ""Brewing Maple Beer at Home"", ""autho...",2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,6189a121-a840-4644-b045-2bae6280cdfd,https://byo.com/article/maple-beer/,NaN,Brewing Maple Beer at Home - Brew Your Own,609.0,10.0,85.0,49.0,12/11/20 3:32,12/11/20 3:32
3,3,3,55,adcc583b-c354-4e5d-ba77-21aa57362a57,https://byo.com/articles/,Search Performed,https://www.google.com/,https://byo.com/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,"{""term"": ""Quad"", ""logged_in"": ""false""}",2019/10/24 6:20:20 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,d01b6f8d-8b54-406c-9483-fb56cf7374ef,https://byo.com/articles/,NaN,Articles - Brew Your Own,640.0,9.0,150.0,43.0,12/11/20 6:31,12/11/20 6:31
4,4,4,57,adcc583b-c354-4e5d-ba77-21aa57362a57,https://byo.com/?s=Quad&post_type=article&orde...,Search Performed,https://www.google.com/,https://byo.com/articles/,Mozilla/5.0 (Linux; Android 9; SM-G960F) Apple...,4:39:01 AM,"{""term"": ""Quad"", ""logged_in"": ""false""}",2019/10/24 6:20:26 PM,t,1,0,0,0,0,f,1,5,0,1,4,4,0,2,0,0,0,0,0,0,0,0,0,0,0,8506d0d9-e138-4800-a33e-de8e48024294,https://byo.com/?s=Quad&post_type=article&orde...,NaN,You searched for Quad - Brew Your Own,770.0,11.0,85.0,43.0,10/24/19 18:20,10/24/19 18:20


In [ ]:
ranking.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'id_x','properties','id_y','current_url','url','title','inserted_at_y'],inplace=True)

In [ ]:
ranking = ranking.rename(columns={'initial_referrerTrue':'initial_referrer','inserted_at_x':'inserted_at'})

# User Loyalty




#1.   Visit frequency = total events/time period

In [ ]:
frequency = ranking.groupby(by='distinct_id')['performed_at'].agg(['count']).reset_index()

In [ ]:
frequency.rename(columns={'count':'frequency'},inplace=True)

In [ ]:
ranking = ranking.merge(frequency,how='inner',on='distinct_id')

In [ ]:
ranking.shape

(1239509, 40)

In [ ]:
ranking['frequency'].value_counts()

358538    358538
1         203998
2          79662
3          47586
45221      45221
           ...  
197          197
195          195
190          190
173          173
160          160
Name: frequency, Length: 328, dtype: int64

In [ ]:
# Min-Max transformation of frequency 
# ranking_user_frequency= ranking.assign(Frequency = ranking.groupby('User_Segment').transform(lambda x: (x - x.min()) / (x.max()- x.min())))

In [ ]:
ranking['inserted_date'] = pd.to_datetime(ranking['inserted_at']).dt.strftime('%Y-%m-%d')
earliest_date = pd.DataFrame(ranking.groupby(['distinct_id'])['inserted_date'].agg(['min']))
ranking = ranking.merge(earliest_date,how='left',on='distinct_id')

In [ ]:
import datetime as dt
today=dt.datetime(2021,3,14)
ranking['min']=pd.to_datetime(ranking['min'],yearfirst=True)

In [ ]:
ranking['User Age'] = pd.DataFrame(today - ranking['min'])

In [ ]:
ranking['User Age'] = ranking['User Age'] / np.timedelta64(1, 'D')

In [ ]:
ranking.head()

,distinct_id,name,initial_referrer,referrer,user_agent,performed_at,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,published_at,dom_count,image_count,link_count,scripts_count,updated_at,frequency,inserted_date,min,User Age
0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00 AM,2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-24,2019-10-24,507.0
1,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01 AM,2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,609.0,10.0,85.0,49.0,12/11/20 3:32,120,2019-10-24,2019-10-24,507.0
2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:18:09 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-25,2019-10-24,507.0
3,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:21:28 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-25,2019-10-24,507.0
4,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:41:10 AM,2019/10/26 6:07:25 AM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-26,2019-10-24,507.0


In [ ]:
ranking['Visit Frequency'] = ranking['frequency']/ranking['User Age']

In [ ]:
ranking['Visit Frequency']

0          0.236686
1          0.236686
2          0.236686
3          0.236686
4          0.236686
             ...   
1239504    0.044118
1239505    0.044118
1239506    0.014706
1239507    0.014706
1239508    0.014706
Name: Visit Frequency, Length: 1239509, dtype: float64


#2.    Most recent visit period


In [ ]:
ranking.reset_index(inplace=True)

In [ ]:
ranking

,index,distinct_id,name,initial_referrer,referrer,user_agent,performed_at,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,published_at,dom_count,image_count,link_count,scripts_count,updated_at,frequency,inserted_date,min,User Age,Visit Frequency
0,0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00 AM,2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-24,2019-10-24,507.0,0.236686
1,1,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01 AM,2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,609.0,10.0,85.0,49.0,12/11/20 3:32,120,2019-10-24,2019-10-24,507.0,0.236686
2,2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:18:09 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-25,2019-10-24,507.0,0.236686
3,3,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:21:28 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-25,2019-10-24,507.0,0.236686
4,4,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:41:10 AM,2019/10/26 6:07:25 AM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-26,2019-10-24,507.0,0.236686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1239504,1239504,4931fdc0-20f8-416b-a039-e27b9090b87a,Viewed Homepage,https://www.google.com,https://byo.com/my-account/?wcm_redirect_to=po...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,3:10:19 PM,2021/01/05 3:59:43 AM,f,1,0,0,0,0,f,1,0,0,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,NaN,672.0,19.0,105.0,48.0,12/10/20 16:33,3,2021-01-05,2021-01-05,68.0,0.044118
1239505,1239505,4931fdc0-20f8-416b-a039-e27b9090b87a,Viewed Homepage,https://www.google.com,NaN,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,3:10:18 PM,2021/01/05 3:56:10 AM,f,1,0,0,0,0,f,1,0,0,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,NaN,672.0,19.0,105.0,48.0,12/10/20 16:33,3,2021-01-05,2021-01-05,68.0,0.044118
1239506,1239506,2da57648-0627-4a09-a6a9-d24f2d8c9baa,Resources Lead Gen,https://bing.com/,https://bing.com/,Mozilla/5.0 (iPhone; CPU iPhone OS 14_2 like M...,3:10:17 PM,2021/01/05 3:31:14 AM,f,0,0,0,0,0,t,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,1928.0,3.0,86.0,47.0,12/10/20 16:33,1,2021-01-05,2021-01-05,68.0,0.014706
1239507,1239507,1d70f5d2-e6fe-4ea5-813d-0adfb3ad369f,Nano Newsletter Subscribe Toaster,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:8...,3:10:17 PM,2021/01/05 3:30:17 AM,f,0,0,0,0,0,t,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,569.0,9.0,83.0,49.0,12/10/20 17:21,1,2021-01-05,2021-01-05,68.0,0.014706


In [ ]:
visit_time = ranking.groupby(by='distinct_id')['index'].agg(['count']).reset_index()

In [ ]:
visit_time

,distinct_id,count
0,000028db-5119-4edd-a73c-b2441e626c65,1
1,0000336c-fa7f-463f-8080-8d59288c3ae5,8
2,000037df-a0f8-4fab-971b-155b548cac9e,1
3,0000d24c-e8af-4de2-8792-ad2269315785,1
4,00011087-656b-4e66-bfd5-1a6725b27c5c,1
...,...,...
293840,ffff165d-fcbb-4227-9c39-2ccab69d9a63,1
293841,ffff33f7-0e6c-4119-a3fc-7c54dd32196a,1
293842,ffff53ac-5d34-436c-8baa-8f931dab1dcc,1
293843,ffff6d60-7ba1-4b22-8b23-70130b506ec5,7


In [ ]:
latest_date = pd.DataFrame(ranking.groupby(['distinct_id'])['inserted_date'].agg(['max']))
ranking = ranking.merge(latest_date,how='left',on='distinct_id')

In [ ]:
ranking

,index,distinct_id,name,initial_referrer,referrer,user_agent,performed_at,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,published_at,dom_count,image_count,link_count,scripts_count,updated_at,frequency,inserted_date,min,User Age,Visit Frequency,max
0,0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00 AM,2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18
1,1,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01 AM,2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,609.0,10.0,85.0,49.0,12/11/20 3:32,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18
2,2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:18:09 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18
3,3,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:21:28 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18
4,4,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:41:10 AM,2019/10/26 6:07:25 AM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-26,2019-10-24,507.0,0.236686,2020-10-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1239504,1239504,4931fdc0-20f8-416b-a039-e27b9090b87a,Viewed Homepage,https://www.google.com,https://byo.com/my-account/?wcm_redirect_to=po...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,3:10:19 PM,2021/01/05 3:59:43 AM,f,1,0,0,0,0,f,1,0,0,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,NaN,672.0,19.0,105.0,48.0,12/10/20 16:33,3,2021-01-05,2021-01-05,68.0,0.044118,2021-01-05
1239505,1239505,4931fdc0-20f8-416b-a039-e27b9090b87a,Viewed Homepage,https://www.google.com,NaN,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,3:10:18 PM,2021/01/05 3:56:10 AM,f,1,0,0,0,0,f,1,0,0,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,NaN,672.0,19.0,105.0,48.0,12/10/20 16:33,3,2021-01-05,2021-01-05,68.0,0.044118,2021-01-05
1239506,1239506,2da57648-0627-4a09-a6a9-d24f2d8c9baa,Resources Lead Gen,https://bing.com/,https://bing.com/,Mozilla/5.0 (iPhone; CPU iPhone OS 14_2 like M...,3:10:17 PM,2021/01/05 3:31:14 AM,f,0,0,0,0,0,t,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,1928.0,3.0,86.0,47.0,12/10/20 16:33,1,2021-01-05,2021-01-05,68.0,0.014706,2021-01-05
1239507,1239507,1d70f5d2-e6fe-4ea5-813d-0adfb3ad369f,Nano Newsletter Subscribe Toaster,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:8...,3:10:17 PM,2021/01/05 3:30:17 AM,f,0,0,0,0,0,t,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,569.0,9.0,83.0,49.0,12/10/20 17:21,1,2021-01-05,2021-01-05,68.0,0.014706,2021

In [ ]:
visit_more_than_once = ranking[ranking['frequency']>1]

In [ ]:
visit_more_than_once

,index,distinct_id,name,initial_referrer,referrer,user_agent,performed_at,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,published_at,dom_count,image_count,link_count,scripts_count,updated_at,frequency,inserted_date,min,User Age,Visit Frequency,max
0,0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00 AM,2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18
1,1,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01 AM,2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,609.0,10.0,85.0,49.0,12/11/20 3:32,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18
2,2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:18:09 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18
3,3,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:21:28 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18
4,4,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:41:10 AM,2019/10/26 6:07:25 AM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-26,2019-10-24,507.0,0.236686,2020-10-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1239471,1239471,9bcea02d-6ea9-4e10-9273-cbfffabe4b5c,Viewed Homepage,https://www.google.com.au,https://byo.com/my-account/?wcm_redirect_to=po...,Mozilla/5.0 (iPhone; CPU iPhone OS 14_3 like M...,3:10:12 PM,2021/01/05 2:02:23 AM,f,1,0,0,0,1,t,1,0,0,0,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,672.0,19.0,105.0,48.0,12/10/20 16:33,2,2021-01-05,2021-01-05,68.0,0.029412,2021-01-05
1239472,1239472,9bcea02d-6ea9-4e10-9273-cbfffabe4b5c,Search Performed,https://www.google.com.au,https://byo.com/my-account/?wcm_redirect_to=po...,Mozilla/5.0 (iPhone; CPU iPhone OS 14_3 like M...,3:10:12 PM,2021/01/05 2:02:24 AM,f,1,0,0,0,1,t,1,0,0,0,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,672.0,19.0,105.0,48.0,12/10/20 16:33,2,2021-01-05,2021-01-05,68.0,0.029412,2021-01-05
1239503,1239503,4931fdc0-20f8-416b-a039-e27b9090b87a,Viewed Homepage,https://www.google.com,https://www.google.com,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,3:10:18 PM,2021/01/05 3:55:22 AM,f,1,0,0,0,0,f,1,0,0,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,NaN,672.0,19.0,105.0,48.0,12/10/20 16:33,3,2021-01-05,2021-01-05,68.0,0.044118,2021-01-05
1239504,1239504,4931fdc0-20f8-416b-a039-e27b9090b87a,Viewed Homepage,https://www.google.com,https://byo.com/my-account/?wcm_redirect_to=po...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,3:10:19 PM,2021/01/05 3:59:43 AM,f,1,0,0,0,0,f,1,0,0,0,3,3,0,3,0,0,0,0,0,0,0,0,0,0,0,NaN,672.0,19.

In [ ]:
sec = visit_more_than_once.copy()

In [ ]:
sec['last_inserted_date'] = sec['max'].astype('datetime64')

In [ ]:
sec.last_inserted_date.dtype

dtype('<M8[ns]')

In [ ]:
sec['last_inserted_date'] = sec['last_inserted_date'].dt.strftime('%Y-%m-%d')

In [ ]:
sec['last_inserted_date']

0          2020-10-18
1          2020-10-18
2          2020-10-18
3          2020-10-18
4          2020-10-18
              ...    
1239471    2021-01-05
1239472    2021-01-05
1239503    2021-01-05
1239504    2021-01-05
1239505    2021-01-05
Name: last_inserted_date, Length: 1035511, dtype: object

In [ ]:
last_inserted_exclude = sec[sec['inserted_date']!=sec['last_inserted_date']]

In [ ]:
last_inserted_exclude

,index,distinct_id,name,initial_referrer,referrer,user_agent,performed_at,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,published_at,dom_count,image_count,link_count,scripts_count,updated_at,frequency,inserted_date,min,User Age,Visit Frequency,max,last_inserted_date
0,0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00 AM,2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18,2020-10-18
1,1,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01 AM,2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,609.0,10.0,85.0,49.0,12/11/20 3:32,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18,2020-10-18
2,2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:18:09 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18,2020-10-18
3,3,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:21:28 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18,2020-10-18
4,4,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:41:10 AM,2019/10/26 6:07:25 AM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-26,2019-10-24,507.0,0.236686,2020-10-18,2020-10-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238287,1238287,6c34f2b8-aee3-4285-9c2b-54f3b27b0e3f,Viewed Homepage,NaN,NaN,Mozilla/5.0 (iPhone; CPU iPhone OS 14_2 like M...,3:08:46 PM,2021/01/04 2:21:00 AM,f,1,0,0,0,0,f,1,0,0,0,2,2,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,672.0,19.0,105.0,48.0,12/10/20 16:33,2,2021-01-04,2021-01-04,69.0,0.028986,2021-01-05,2021-01-05
1238879,1238879,c128a3e1-16a0-4dab-84aa-dab57046c137,Viewed Homepage,https://www.google.com/,https://byo.com/product/gift-combo-digital-mem...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,3:09:45 PM,2021/01/04 6:43:50 PM,t,0,0,0,0,0,f,0,0,0,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,672.0,19.0,105.0,48.0,12/10/20 16:33,3,2021-01-04,2021-01-04,69.0,0.043478,2021-01-05,2021-01-05
1238880,1238880,c128a3e1-16a0-4dab-84aa-dab57046c137,Viewed Homepage,https://www.google.com/,https://byo.com/article/beyond-the-barrel/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,3:09:45 PM,2021/01/04 6:33:43 PM,t,0,0,0,0,0,f,0,0,0,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,NaN,672.0,19.0,105.0,48.0,12/10/20 16:33,3,2021-01-04,2021-01-04,69.0,0.043478,2021-01-05,2021-01-05
1239017,1239017,f6061c2f-3205-4c64-abaa-cb2eba6c82f4,New England IPA Lead Gen with Guide,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,3:09:46 PM,2021/01/04 6:51:23 PM,f,1,0,0,0

In [ ]:
last_inserted_exclude.shape

(841198, 47)

In [ ]:
second_latest_date = pd.DataFrame(last_inserted_exclude.groupby(['distinct_id'])['inserted_date'].agg(['max']))

In [ ]:
second_latest_date = second_latest_date.reset_index()

In [ ]:
second_latest_date.rename(columns={'max':'second_latest_date'},inplace=True)

In [ ]:
last_inserted_exclude = last_inserted_exclude.merge(second_latest_date,on='distinct_id')

In [ ]:
last_inserted_exclude.shape

(841198, 48)

In [ ]:
visit_more_than_once.shape

(1035511, 46)

In [ ]:
sec_last_col = last_inserted_exclude[['distinct_id','last_inserted_date','second_latest_date']]

In [ ]:
sec_last_col.shape

(841198, 3)

In [ ]:
sec_last_col

,distinct_id,last_inserted_date,second_latest_date
0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,2020-10-18,2020-10-14
1,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,2020-10-18,2020-10-14
2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,2020-10-18,2020-10-14
3,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,2020-10-18,2020-10-14
4,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,2020-10-18,2020-10-14
...,...,...,...
841193,6c34f2b8-aee3-4285-9c2b-54f3b27b0e3f,2021-01-05,2021-01-04
841194,c128a3e1-16a0-4dab-84aa-dab57046c137,2021-01-05,2021-01-04
841195,c128a3e1-16a0-4dab-84aa-dab57046c137,2021-01-05,2021-01-04
841196,f6061c2f-3205-4c64-abaa-cb2eba6c82f4,2021-01-05,2021-01-04


In [ ]:
sec_last_col.drop_duplicates(inplace=True, keep='last')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
sec_last_col

,distinct_id,last_inserted_date,second_latest_date
118,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,2020-10-18,2020-10-14
242,adcc583b-c354-4e5d-ba77-21aa57362a57,2020-04-30,2020-03-29
484,dfe9832f-52f4-4e9f-8cc9-3da4bf4e30d1,2020-08-25,2020-08-19
724,d41f78b5-2b70-4cac-a008-503f09abf73b,2020-06-24,2020-06-03
947,2078a4ac-770e-402c-a8e8-9c996dc631af,2020-05-17,2020-05-08
...,...,...,...
841192,c126efa9-95bf-4576-b608-c1aed8ff5b83,2021-01-05,2021-01-04
841193,6c34f2b8-aee3-4285-9c2b-54f3b27b0e3f,2021-01-05,2021-01-04
841195,c128a3e1-16a0-4dab-84aa-dab57046c137,2021-01-05,2021-01-04
841196,f6061c2f-3205-4c64-abaa-cb2eba6c82f4,2021-01-05,2021-01-04


In [ ]:
last_inserted = sec[sec['inserted_date']==sec['last_inserted_date']]

In [ ]:
last_inserted.shape

(194313, 47)

In [ ]:
841198 + 194313

1035511

In [ ]:
visit_more_than_once.shape

(1035511, 46)

In [ ]:
visit_more_than_once.head()

,index,distinct_id,name,initial_referrer,referrer,user_agent,performed_at,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,published_at,dom_count,image_count,link_count,scripts_count,updated_at,frequency,inserted_date,min,User Age,Visit Frequency,max
0,0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00 AM,2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18
1,1,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01 AM,2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,609.0,10.0,85.0,49.0,12/11/20 3:32,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18
2,2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:18:09 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18
3,3,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:21:28 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18
4,4,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:41:10 AM,2019/10/26 6:07:25 AM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-26,2019-10-24,507.0,0.236686,2020-10-18


In [ ]:
sec_last_col.shape

(56795, 3)

In [ ]:
new_user_excluded = visit_more_than_once.merge(sec_last_col,how='left',on='distinct_id')

In [ ]:
new_user_excluded.shape

(1035511, 48)

In [ ]:
new_user_excluded['last_inserted_date'] = pd.to_datetime(new_user_excluded['last_inserted_date'])

In [ ]:
new_user_excluded['second_latest_date'] = pd.to_datetime(new_user_excluded['second_latest_date'])

In [ ]:
new_user_excluded['recent_visit_period'] = new_user_excluded['last_inserted_date']-new_user_excluded['second_latest_date']

In [ ]:
new_user_excluded['recent_visit_period'] = new_user_excluded['recent_visit_period']/ np.timedelta64(1, 'D')

In [ ]:
new_user_excluded.head()

,index,distinct_id,name,initial_referrer,referrer,user_agent,performed_at,inserted_at,membership_purchased,total_sessions,content_count,paywalls_count,recommended_content_clicks,searches_performed,prior_user,total_events,ecommerce_event_count,account_management_event_count,total_cta_count,total_ugc_count,homepage_viewed_count,product_viewed_count,referrer_count,restricted_content_count,recipe_view_count,mr_wizard_view_count,article_view_count,resource_view_count,cta_impression_count,cta_conversion_count,negative_content_viewed_count,mixed_content_viewed_count,positive_content_viewed_count,neutral_content_viewed_count,published_at,dom_count,image_count,link_count,scripts_count,updated_at,frequency,inserted_date,min,User Age,Visit Frequency,max,last_inserted_date,second_latest_date,recent_visit_period
0,0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:00 AM,2019/10/24 6:16:21 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18,2020-10-18,2020-10-14,4.0
1,1,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:39:01 AM,2019/10/24 6:19:47 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,609.0,10.0,85.0,49.0,12/11/20 3:32,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18,2020-10-18,2020-10-14,4.0
2,2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:18:09 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,521.0,9.0,83.0,50.0,9/26/20 19:10,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18,2020-10-18,2020-10-14,4.0
3,3,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:40:20 AM,2019/10/25 4:21:28 PM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18,2020-10-18,2020-10-14,4.0
4,4,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,Content Viewed,https://www.google.com/,https://www.google.com/,Mozilla/5.0 (Linux; Android 8.0.0; AGS2-W09) A...,4:41:10 AM,2019/10/26 6:07:25 AM,f,1,47,0,0,6,t,1,0,0,0,76,23,0,13,41,11,0,35,0,0,0,0,0,7,33,NaN,507.0,6.0,82.0,50.0,11/30/20 22:28,120,2019-10-26,2019-10-24,507.0,0.236686,2020-10-18,2020-10-18,2020-10-14,4.0


In [ ]:
ranking.columns

Index(['index', 'distinct_id', 'name', 'initial_referrer', 'referrer',
       'user_agent', 'performed_at', 'inserted_at', 'membership_purchased',
       'total_sessions', 'content_count', 'paywalls_count',
       'recommended_content_clicks', 'searches_performed', 'prior_user',
       'total_events', 'ecommerce_event_count',
       'account_management_event_count', 'total_cta_count', 'total_ugc_count',
       'homepage_viewed_count', 'product_viewed_count', 'referrer_count',
       'restricted_content_count', 'recipe_view_count', 'mr_wizard_view_count',
       'article_view_count', 'resource_view_count', 'cta_impression_count',
       'cta_conversion_count', 'negative_content_viewed_count',
       'mixed_content_viewed_count', 'positive_content_viewed_count',
       'neutral_content_viewed_count', 'published_at', 'dom_count',
       'image_count', 'link_count', 'scripts_count', 'updated_at', 'frequency',
       'inserted_date', 'min', 'User Age', 'Visit Frequency', 'max'],
      dtype

In [ ]:
ranking_with_loyalty = ranking[['distinct_id','performed_at', 'inserted_at', 
                   'frequency','inserted_date', 'User Age', 'Visit Frequency',
                   'membership_purchased','prior_user','referrer_count','recommended_content_clicks', 
                   'searches_performed', 'ecommerce_event_count','account_management_event_count', 'total_ugc_count', 
                   'total_cta_count', 'cta_impression_count','cta_conversion_count']]

In [ ]:
ranking_with_loyalty['last_inserted_date'] = np.nan
ranking_with_loyalty['second_latest_date'] = np.nan
ranking_with_loyalty['recent_visit_period']= np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
ranking_with_loyalty.head()

,distinct_id,performed_at,inserted_at,frequency,inserted_date,User Age,Visit Frequency,membership_purchased,prior_user,referrer_count,recommended_content_clicks,searches_performed,ecommerce_event_count,account_management_event_count,total_ugc_count,total_cta_count,cta_impression_count,cta_conversion_count,last_inserted_date,second_latest_date,recent_visit_period
0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,4:39:00 AM,2019/10/24 6:16:21 PM,120,2019-10-24,507.0,0.236686,f,t,13,0,6,0,0,76,0,0,0,NaN,NaN,NaN
1,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,4:39:01 AM,2019/10/24 6:19:47 PM,120,2019-10-24,507.0,0.236686,f,t,13,0,6,0,0,76,0,0,0,NaN,NaN,NaN
2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,4:40:20 AM,2019/10/25 4:18:09 PM,120,2019-10-25,507.0,0.236686,f,t,13,0,6,0,0,76,0,0,0,NaN,NaN,NaN
3,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,4:40:20 AM,2019/10/25 4:21:28 PM,120,2019-10-25,507.0,0.236686,f,t,13,0,6,0,0,76,0,0,0,NaN,NaN,NaN
4,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,4:41:10 AM,2019/10/26 6:07:25 AM,120,2019-10-26,507.0,0.236686,f,t,13,0,6,0,0,76,0,0,0,NaN,NaN,NaN


In [ ]:
ranking_with_loyalty = ranking_with_loyalty[['distinct_id','performed_at', 'inserted_at', 
                   'frequency','inserted_date', 'User Age', 'Visit Frequency','last_inserted_date', 'second_latest_date', 'recent_visit_period',
                   'membership_purchased','prior_user','referrer_count','recommended_content_clicks', 
                   'searches_performed', 'ecommerce_event_count','account_management_event_count', 'total_ugc_count', 
                   'total_cta_count', 'cta_impression_count','cta_conversion_count']]

In [ ]:
new_user = ranking_with_loyalty[~ranking_with_loyalty['distinct_id'].isin(new_user_excluded['distinct_id'])]

In [ ]:
new_user.shape

(203998, 21)

In [ ]:
new_user['New/Old user'] = 'New'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
new_user_excluded.columns

Index(['index', 'distinct_id', 'name', 'initial_referrer', 'referrer',
       'user_agent', 'performed_at', 'inserted_at', 'membership_purchased',
       'total_sessions', 'content_count', 'paywalls_count',
       'recommended_content_clicks', 'searches_performed', 'prior_user',
       'total_events', 'ecommerce_event_count',
       'account_management_event_count', 'total_cta_count', 'total_ugc_count',
       'homepage_viewed_count', 'product_viewed_count', 'referrer_count',
       'restricted_content_count', 'recipe_view_count', 'mr_wizard_view_count',
       'article_view_count', 'resource_view_count', 'cta_impression_count',
       'cta_conversion_count', 'negative_content_viewed_count',
       'mixed_content_viewed_count', 'positive_content_viewed_count',
       'neutral_content_viewed_count', 'published_at', 'dom_count',
       'image_count', 'link_count', 'scripts_count', 'updated_at', 'frequency',
       'inserted_date', 'min', 'User Age', 'Visit Frequency', 'max',
       'last

In [ ]:
new_user_excluded = new_user_excluded[['distinct_id','performed_at', 'inserted_at', 
                                       'frequency','inserted_date', 'User Age', 'Visit Frequency', 'last_inserted_date', 'second_latest_date', 'recent_visit_period',
                                       'membership_purchased','prior_user','referrer_count','recommended_content_clicks', 
                                        'searches_performed', 'ecommerce_event_count','account_management_event_count', 'total_ugc_count', 
                                        'total_cta_count', 'cta_impression_count','cta_conversion_count']]

In [ ]:
new_user_excluded['New/Old user'] = 'Recurrent'

In [ ]:
new_user_excluded['last_inserted_date'] = new_user_excluded['last_inserted_date'].astype('datetime64')

In [ ]:
ranking_with_loyalty = pd.concat([new_user,new_user_excluded])

In [ ]:
ranking_with_loyalty['last_inserted_date'].dtype

dtype('<M8[ns]')

In [ ]:
ranking_with_loyalty['last_inserted_date'] = ranking_with_loyalty['last_inserted_date'].dt.strftime('%Y-%m-%d')

In [ ]:
ranking_with_loyalty['second_latest_date'] = ranking_with_loyalty['second_latest_date'].dt.strftime('%Y-%m-%d')

In [ ]:
ranking_with_loyalty['last_inserted_date'].fillna(ranking_with_loyalty['inserted_date'],inplace=True)

In [ ]:
ranking_with_loyalty['last_inserted_date'].isna().sum()

0

In [ ]:
ranking_with_loyalty.head()

,distinct_id,performed_at,inserted_at,frequency,inserted_date,User Age,Visit Frequency,last_inserted_date,second_latest_date,recent_visit_period,membership_purchased,prior_user,referrer_count,recommended_content_clicks,searches_performed,ecommerce_event_count,account_management_event_count,total_ugc_count,total_cta_count,cta_impression_count,cta_conversion_count,New/Old user
542765,c0f06d38-0169-4d6b-8157-86a8b7edd5b8,7:03:04 AM,2020/02/01 7:11:25 PM,1,2020-02-01,407.0,0.002457,2020-02-01,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New
551448,6b170cb4-3c8a-4b63-ab8b-4a60fc2ec2fe,7:16:45 AM,2020/02/11 7:14:10 AM,1,2020-02-11,397.0,0.002519,2020-02-11,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New
652381,3312de31-bf96-4448-b9c1-881faec9149d,8:35:22 AM,2020/04/05 9:38:37 PM,1,2020-04-05,343.0,0.002915,2020-04-05,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New
688570,7f2f5839-55fc-4f34-9dc1-bd5b082f8ba2,9:16:49 AM,2020/05/04 4:32:35 PM,1,2020-05-04,314.0,0.003185,2020-05-04,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New
715833,7bb22bbb-0c2e-43c5-b9ad-1479b301c7bc,10:07:54 AM,2020/06/09 3:52:42 AM,1,2020-06-09,278.0,0.003597,2020-06-09,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New


In [ ]:
#ranking_with_loyalty.to_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/ranking_with_loyalty.csv') 

# Promotion Effect
#1. New User after XXXX-XX-XX

In [ ]:
# How many users log in after 2020-12-31? -> Active Users

In [ ]:
test = ranking_with_loyalty[ranking_with_loyalty['last_inserted_date']>'2020-12-31']

In [ ]:
test['distinct_id'].nunique()

4159

In [ ]:
test.columns

Index(['distinct_id', 'performed_at', 'inserted_at', 'frequency',
       'inserted_date', 'User Age', 'Visit Frequency', 'last_inserted_date',
       'second_latest_date', 'recent_visit_period', 'membership_purchased',
       'prior_user', 'referrer_count', 'recommended_content_clicks',
       'searches_performed', 'ecommerce_event_count',
       'account_management_event_count', 'total_ugc_count', 'total_cta_count',
       'cta_impression_count', 'cta_conversion_count', 'New/Old user'],
      dtype='object')

In [ ]:
test['active_user']='t'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
active_user = test[['distinct_id','active_user']]

In [ ]:
active_user.drop_duplicates(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
ranking_with_promotion_effect = ranking_with_loyalty.merge(active_user,how='left',on='distinct_id')

In [ ]:
ranking_with_promotion_effect.head()

,distinct_id,performed_at,inserted_at,frequency,inserted_date,User Age,Visit Frequency,last_inserted_date,second_latest_date,recent_visit_period,membership_purchased,prior_user,referrer_count,recommended_content_clicks,searches_performed,ecommerce_event_count,account_management_event_count,total_ugc_count,total_cta_count,cta_impression_count,cta_conversion_count,New/Old user,active_user
0,c0f06d38-0169-4d6b-8157-86a8b7edd5b8,7:03:04 AM,2020/02/01 7:11:25 PM,1,2020-02-01,407.0,0.002457,2020-02-01,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New,NaN
1,6b170cb4-3c8a-4b63-ab8b-4a60fc2ec2fe,7:16:45 AM,2020/02/11 7:14:10 AM,1,2020-02-11,397.0,0.002519,2020-02-11,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New,NaN
2,3312de31-bf96-4448-b9c1-881faec9149d,8:35:22 AM,2020/04/05 9:38:37 PM,1,2020-04-05,343.0,0.002915,2020-04-05,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New,NaN
3,7f2f5839-55fc-4f34-9dc1-bd5b082f8ba2,9:16:49 AM,2020/05/04 4:32:35 PM,1,2020-05-04,314.0,0.003185,2020-05-04,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New,NaN
4,7bb22bbb-0c2e-43c5-b9ad-1479b301c7bc,10:07:54 AM,2020/06/09 3:52:42 AM,1,2020-06-09,278.0,0.003597,2020-06-09,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New,NaN


In [ ]:
ranking_with_promotion_effect['active_user'].fillna('f',inplace=True)

In [ ]:
# membership:t -> prior user: f -> New Member

In [ ]:
new_member = test[(test['membership_purchased']=='t')&(test['prior_user']=='f')]

In [ ]:
new_member['distinct_id'].nunique()

162

In [ ]:
new_member['new_member'] = 't'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
new_member = new_member[['distinct_id','new_member']]

In [ ]:
new_member.drop_duplicates(inplace=True)

In [ ]:
ranking_with_promotion_effect = ranking_with_promotion_effect.merge(new_member,how='left',on='distinct_id')

In [ ]:
ranking_with_promotion_effect['new_member'].fillna('f',inplace=True)

In [ ]:
ranking_with_promotion_effect

,distinct_id,performed_at,inserted_at,frequency,inserted_date,User Age,Visit Frequency,last_inserted_date,second_latest_date,recent_visit_period,membership_purchased,prior_user,referrer_count,recommended_content_clicks,searches_performed,ecommerce_event_count,account_management_event_count,total_ugc_count,total_cta_count,cta_impression_count,cta_conversion_count,New/Old user,active_user,new_member
0,c0f06d38-0169-4d6b-8157-86a8b7edd5b8,7:03:04 AM,2020/02/01 7:11:25 PM,1,2020-02-01,407.0,0.002457,2020-02-01,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New,f,f
1,6b170cb4-3c8a-4b63-ab8b-4a60fc2ec2fe,7:16:45 AM,2020/02/11 7:14:10 AM,1,2020-02-11,397.0,0.002519,2020-02-11,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New,f,f
2,3312de31-bf96-4448-b9c1-881faec9149d,8:35:22 AM,2020/04/05 9:38:37 PM,1,2020-04-05,343.0,0.002915,2020-04-05,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New,f,f
3,7f2f5839-55fc-4f34-9dc1-bd5b082f8ba2,9:16:49 AM,2020/05/04 4:32:35 PM,1,2020-05-04,314.0,0.003185,2020-05-04,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New,f,f
4,7bb22bbb-0c2e-43c5-b9ad-1479b301c7bc,10:07:54 AM,2020/06/09 3:52:42 AM,1,2020-06-09,278.0,0.003597,2020-06-09,NaN,NaN,f,t,1,0,1,0,0,1,0,0,0,New,f,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1239504,9bcea02d-6ea9-4e10-9273-cbfffabe4b5c,3:10:12 PM,2021/01/05 2:02:23 AM,2,2021-01-05,68.0,0.029412,2021-01-05,NaN,NaN,f,t,1,0,1,0,0,2,0,0,0,Recurrent,t,f
1239505,9bcea02d-6ea9-4e10-9273-cbfffabe4b5c,3:10:12 PM,2021/01/05 2:02:24 AM,2,2021-01-05,68.0,0.029412,2021-01-05,NaN,NaN,f,t,1,0,1,0,0,2,0,0,0,Recurrent,t,f
1239506,4931fdc0-20f8-416b-a039-e27b9090b87a,3:10:18 PM,2021/01/05 3:55:22 AM,3,2021-01-05,68.0,0.044118,2021-01-05,NaN,NaN,f,f,3,0,0,0,0,3,0,0,0,Recurrent,t,f
1239507,4931fdc0-20f8-416b-a039-e27b9090b87a,3:10:19 PM,2021/01/05 3:59:43 AM,3,2021-01-05,68.0,0.044118,2021-01-05,NaN,NaN,f,f,3,0,0,0,0,3,0,0,0,Recurrent,t,f


In [ ]:
ranking_clean = ranking_with_promotion_effect.copy()

In [ ]:
ranking_clean.to_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/ranking_clean.csv')

# Second last date

In [ ]:
vmto = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/visit_more_than_once.csv')
scl = pd.read_csv('/content/drive/Shareddrives/Sterling Woods Data Challenge/Data set/sec_last_col.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
vmtoo = vmto[['distinct_id','frequency','inserted_date','min','User Age','Visit Frequency','max']]

In [ ]:
vmtoo.shape

(1035511, 7)

In [ ]:
scl = scl[['distinct_id','second_latest_date']]

In [ ]:
scl.drop_duplicates(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
ss = vmtoo.merge(scl,on='distinct_id')

In [ ]:
ss

,distinct_id,frequency,inserted_date,min,User Age,Visit Frequency,max,second_latest_date
0,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18,2020-10-14
1,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,120,2019-10-24,2019-10-24,507.0,0.236686,2020-10-18,2020-10-14
2,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18,2020-10-14
3,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,120,2019-10-25,2019-10-24,507.0,0.236686,2020-10-18,2020-10-14
4,7f9a1331-4aa5-40f8-81a0-875ba9dc3d5f,120,2019-10-26,2019-10-24,507.0,0.236686,2020-10-18,2020-10-14
...,...,...,...,...,...,...,...,...
933836,f6061c2f-3205-4c64-abaa-cb2eba6c82f4,3,2021-01-04,2021-01-04,69.0,0.043478,2021-01-05,2021-01-04
933837,f6061c2f-3205-4c64-abaa-cb2eba6c82f4,3,2021-01-05,2021-01-04,69.0,0.043478,2021-01-05,2021-01-04
933838,f6061c2f-3205-4c64-abaa-cb2eba6c82f4,3,2021-01-05,2021-01-04,69.0,0.043478,2021-01-05,2021-01-04
933839,9a7d9c0e-bb9f-4c80-957a-1a704cc0c6d4,2,2021-01-04,2021-01-04,69.0,0.028986,2021-01-05,2021-01-04
